<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
<p>Written by: Xavier</p>

<h2>Import Packages</h2>

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
print("Imported packages...")

Imported packages...


<h2>Section 1: Initialize Beautiful Soup & Create DataFrame</h2>

In [2]:
# Get data from Wikipedia
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
print("Got Wikipedia data...")

# Parse using BeautifulSoup
soup = BeautifulSoup(r,'lxml')
print("Parsed Wikipedia data...")

# Empty list to store data later
table_contents = []

# Get all tables on the Wikipedia page
# KEY_NOTE: There's only one table
table = soup.find('table')
print("Found table...")

# Transpose rows in Wikipedia table
# to empty list
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# Use newly updated list to create dataframe
# with correct headings and format
df=pd.DataFrame(table_contents)
print("Created dataframe...")
df['Borough']=df['Borough'].replace(
    {
        'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
        'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
        'EtobicokeNorthwest':'Etobicoke Northwest',
        'East YorkEast Toronto':'East York/East Toronto',
        'MississaugaCanada Post Gateway Processing Centre':'Mississauga'
    }
)
print("Formatted dataframe...")

Got Wikipedia data...
Parsed Wikipedia data...
Found table...
Created dataframe...
Formatted dataframe...


In [3]:
# Check if data is formatted correctly
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [4]:
# Check the size of the dataframe
df.shape

(103, 3)

<h2>Section 2: Get Lat. & Long.</h2>

In [5]:
geo_data = pd.read_csv("Geospatial_Coordinates.csv")
geo_data.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df = pd.merge(geo_data, df, how='right', on='PostalCode')

In [7]:
df.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M3A,43.753259,-79.329656,North York,Parkwoods
1,M4A,43.725882,-79.315572,North York,Victoria Village
2,M5A,43.654260,-79.360636,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,43.718518,-79.464763,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,43.662301,-79.389494,Queen's Park,Ontario Provincial Government


<h2>Section 3: Clustering</h2>

<h3>Credentials</h3>

In [8]:
CLIENT_ID = '4LLKEHMGJNMAFBMZWOJHAJCAGIIZ2QTCLDXYAWDHHLTUDNRW'
CLIENT_SECRET = 'XNS0QALDCCD1UH5FZ3B4N1SJJISVBPM2J4DOZHL1UNODPW5A'
VERSION = '20210501'
LIMIT = 100

<h3>Let's create a function to repeat the same process to all the neighborhoods in Toronto</h3>

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3>Now run the above function on each neighborhood and create a new dataframe called toronto_venues.</h3>

In [10]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

<h3>Let's check the size of the resulting dataframe</h3>

In [11]:
print(toronto_venues.shape)
toronto_venues.head()

(1992, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


<h3>Let's check how many venues were returned for each neighborhood</h3>

In [12]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",2,2,2,2,2,2
Woburn,4,4,4,4,4,4
Woodbine Heights,5,5,5,5,5,5


In [13]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 256 uniques categories.


<h2>Analyze Each Neighborhood</h2>

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>And let's examine the new dataframe size.</h3>

In [15]:
toronto_onehot.shape

(1992, 256)

<h3>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h3>

In [16]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3>Let's confirm the new size</h3>

In [17]:
toronto_grouped.shape

(101, 256)

<h3>Let's print each neighborhood along with the top 5 most common venues</h3>

In [18]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

07
2       Airport Lounge  0.07
3     Sculpture Garden  0.07
4          Coffee Shop  0.07


----Caledonia-Fairbanks----
               venue  freq
0               Park  0.50
1      Women's Store  0.25
2               Pool  0.25
3        Men's Store  0.00
4  Mobile Phone Shop  0.00


----Cedarbrae----
                  venue  freq
0                Bakery  0.12
1  Caribbean Restaurant  0.12
2                Lounge  0.12
3      Hakka Restaurant  0.12
4                  Bank  0.12


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1      Sandwich Place  0.08
2    Sushi Restaurant  0.06
3                Café  0.05
4  Italian Restaurant  0.05


----Christie----
           venue  freq
0  Grocery Store  0.29
1           Café  0.21
2           Park  0.14
3    Coffee Shop  0.07
4     Baby Store  0.07


----Church and Wellesley----
                  venue  freq
0      Sushi Restaurant  0.09
1   Japanese Restaurant  0.08
2            Restaurant  0.05
3           C

<h3>Let's put that into a _pandas_ dataframe</h3>

In [19]:
# Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
# Create the new dataframe and 
# display the top 10 venues for 
# each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Clothing Store,Skating Rink,Latin American Restaurant,Breakfast Spot,Yoga Studio,Metro Station,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,Vegetarian / Vegan Restaurant,Turkish Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Sandwich Place,Fried Chicken Joint,Sushi Restaurant,Frozen Yogurt Shop,Deli / Bodega,Intersection,Gas Station
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Pizza Place,Indian Restaurant,Pharmacy,Cupcake Shop,Pub,Café,Butcher


<h2>Cluster Neighbourhoods</h2>

<h3>Run _k_-means to cluster the neighborhood into 5 clusters.</h3>

In [21]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

<h3>Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.</h3>

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,43.753259,-79.329656,North York,Parkwoods,0.0,Convenience Store,Park,Fast Food Restaurant,Food & Drink Shop,Yoga Studio,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Medical Center
1,M4A,43.725882,-79.315572,North York,Victoria Village,0.0,Coffee Shop,Hockey Arena,Intersection,Portuguese Restaurant,Pizza Place,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Martial Arts School
2,M5A,43.654260,-79.360636,Downtown Toronto,"Regent Park, Harbourfront",0.0,Coffee Shop,Park,Bakery,Pub,Café,Event Space,Beer Store,French Restaurant,Performing Arts Venue,Breakfast Spot
3,M6A,43.718518,-79.464763,North York,"Lawrence Manor, Lawrence Heights",0.0,Clothing Store,Coffee Shop,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Furniture / Home Store,Park,Organic Grocery,Martial Arts School,Massage Studio
4,M7A,43.662301,-79.389494,Queen's Park,Ontario Provincial Government,0.0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Mexican Restaurant,Burger Joint,Café,College Auditorium,Diner,Japanese Restaurant


<h3>Visualize clusters</h3>

In [23]:
# Map variables
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Convert clusters to in
tr_clusters = toronto_merged.fillna(0)
tr_clusters = tr_clusters['Cluster Labels'].astype(int)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], tr_clusters):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h2>Examine Clusters</h2>

<h3>Cluster 1</h3>

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.753259,0.0,Convenience Store,Park,Fast Food Restaurant,Food & Drink Shop,Yoga Studio,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Medical Center
1,43.725882,0.0,Coffee Shop,Hockey Arena,Intersection,Portuguese Restaurant,Pizza Place,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Martial Arts School
2,43.654260,0.0,Coffee Shop,Park,Bakery,Pub,Café,Event Space,Beer Store,French Restaurant,Performing Arts Venue,Breakfast Spot
3,43.718518,0.0,Clothing Store,Coffee Shop,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Furniture / Home Store,Park,Organic Grocery,Martial Arts School,Massage Studio
4,43.662301,0.0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Mexican Restaurant,Burger Joint,Café,College Auditorium,Diner,Japanese Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
97,43.648429,0.0,Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Asian Restaurant,Restaurant,Steakhouse,Sushi Restaurant,Deli / Bodega
98,43.653654,0.0,River,Smoke Shop,Pool,Molecular Gastronomy Restaurant,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
99,43.665860,0.0,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Fast Food Restaurant,Mediterranean Restaurant,Gay Bar,Gym,Burrito Place,Indian Restaurant
100,43.662744,0.0,Light Rail Station,Farmers Market,Garden Center,Comic Shop,Skate Park,Gym / Fitness Center,Pizza Place,Park,Fast Food Restaurant,Brewery


<h3>Cluster 2</h3>

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,43.689026,1.0,Park,Women's Store,Pool,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Medical Center
35,43.685347,1.0,Park,Intersection,Convenience Store,Metro Station,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Yoga Studio
40,43.737473,1.0,Park,Airport,Yoga Studio,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Medical Center
52,43.789053,1.0,Park,Yoga Studio,Molecular Gastronomy Restaurant,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
64,43.706876,1.0,Park,Convenience Store,Yoga Studio,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Mediterranean Restaurant
66,43.752758,1.0,Park,Convenience Store,Yoga Studio,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Mediterranean Restaurant
77,43.688905,1.0,Park,Sandwich Place,Yoga Studio,Modern European Restaurant,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant
91,43.679563,1.0,Park,Playground,Trail,Yoga Studio,Men's Store,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Medical Center


<h3>Cluster 3</h3>

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,43.724766,2.0,Baseball Field,Yoga Studio,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
101,43.636258,2.0,Baseball Field,Yoga Studio,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store


<h3>Cluster 4</h3>

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,43.689574,3.0,Restaurant,Yoga Studio,Molecular Gastronomy Restaurant,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store


<h3>Cluster 5</h3>

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,43.784535,4.0,Bar,Yoga Studio,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
